In [1]:
# h1 초기 해시값 구하기 

In [1]:
import math
import numpy as np

n = math.sqrt(3) - 1
inx = np.zeros((8, 4))

for i in range(32):
    row = i//4
    remain = i%4
    
    n = n*2
    print(i+1, " - ", n/2, " * 2 = ", n)
    
    if n < 1:
        inx[row][remain] = 0
    else:
        n = n - 1
        inx[row][remain] = 1

inx = inx.flatten()
bit = ""
for i in inx:
    bit += str(int(i))
    
bit = "0b" + bit
n_10 = int(bit, 2)
n_16 = hex(n_10)
print(n_16)

1  -  0.7320508075688772  * 2 =  1.4641016151377544
2  -  0.4641016151377544  * 2 =  0.9282032302755088
3  -  0.9282032302755088  * 2 =  1.8564064605510175
4  -  0.8564064605510175  * 2 =  1.712812921102035
5  -  0.7128129211020351  * 2 =  1.4256258422040702
6  -  0.4256258422040702  * 2 =  0.8512516844081404
7  -  0.8512516844081404  * 2 =  1.7025033688162807
8  -  0.7025033688162807  * 2 =  1.4050067376325615
9  -  0.40500673763256145  * 2 =  0.8100134752651229
10  -  0.8100134752651229  * 2 =  1.6200269505302458
11  -  0.6200269505302458  * 2 =  1.2400539010604916
12  -  0.24005390106049163  * 2 =  0.48010780212098325
13  -  0.48010780212098325  * 2 =  0.9602156042419665
14  -  0.9602156042419665  * 2 =  1.920431208483933
15  -  0.920431208483933  * 2 =  1.840862416967866
16  -  0.840862416967866  * 2 =  1.681724833935732
17  -  0.681724833935732  * 2 =  1.363449667871464
18  -  0.36344966787146404  * 2 =  0.7268993357429281
19  -  0.7268993357429281  * 2 =  1.4537986714858562
20  -

In [ ]:
# SHA256 해시 작동 원리

In [184]:
from bitstring import BitArray

In [185]:
# hash constants K[0] ~ k[63]
constants = (
            0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
            0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
            0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
            0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
            0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
            0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
            0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
            0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
            0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
            0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
            0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
            0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
            0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
            0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
            0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
            0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2)
# h0 ~ h7
h = (
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19)

In [186]:
def Pad(W):
    return bytes(W, "ascii") + b"\x80" + (b"\x00" * ((55 if (len(W) % 64) < 56 else 119) - (len(W) % 64))) + (
        (len(W) << 3).to_bytes(8, "big"))

In [187]:
message = input("Enter string : ")
message = Pad(message) # byte 
print("Padding : \n",BitArray(message).bin) # 바이트를 비트로 

Enter string : i think you are
Padding : 
 01101001001000000111010001101000011010010110111001101011001000000111100101101111011101010010000001100001011100100110010110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001111000


In [188]:
def right_rotate(num, shift):
    # x에서 b 비트만큼 오른쪽으로 이동 
    return ((num >> shift) | (num << (32 - shift)))

In [189]:
# 64개의 message array W[0] ~ W[63] 만들기
# 512 bit == 64 byte 
for i in range(0, len(message), 64):
    # 64바이트 가져오기 == 512비트 가져오기
    S = message[i: i + 64]
    
    # 32비트씩 나누기 => 16개로 나누기 W[0] ~ W[15]
    W = [int.from_bytes(S[e: e + 4], 'big') for e in range(0, 64, 4)] + ([0] * 48)
    
    # W[16] ~ W[63]
    for j in range(16, 64):
        sig0 = (right_rotate(W[j-15], 7) ^ right_rotate(W[j-15], 18) ^ (W[j-15] >> 3))
        sig1 = (right_rotate(W[j-2], 17) ^ right_rotate(W[j-2], 19) ^ (W[j-2] >> 10))
        W[j] = (((W[j-16] + sig0 + W[j-7] + sig1))) & ((2 ** 32) - 1)
        
for idx, w in enumerate(W):
    print("W[{}] = {:032b}".format(idx, w))



W[0] = 01101001001000000111010001101000
W[1] = 01101001011011100110101100100000
W[2] = 01111001011011110111010100100000
W[3] = 01100001011100100110010110000000
W[4] = 00000000000000000000000000000000
W[5] = 00000000000000000000000000000000
W[6] = 00000000000000000000000000000000
W[7] = 00000000000000000000000000000000
W[8] = 00000000000000000000000000000000
W[9] = 00000000000000000000000000000000
W[10] = 00000000000000000000000000000000
W[11] = 00000000000000000000000000000000
W[12] = 00000000000000000000000000000000
W[13] = 00000000000000000000000000000000
W[14] = 00000000000000000000000000000000
W[15] = 00000000000000000000000001111000
W[16] = 01000000010101111000000001010001
W[17] = 11111100001110001001100100110101
W[18] = 00111111001011101100001100001000
W[19] = 11000000111101011011010100111101
W[20] = 10111001111010101101001111000010
W[21] = 01101100000010010110010100001001
W[22] = 00110011101101110111000111110100
W[23] = 11011110100101100001101000101101
W[24] = 010100110000000100

In [190]:
# initialize working variables to current has values
digest = list(h)
A, B, C, D, E, F, G, H = digest

In [191]:
# compression
def compression(Wt, Kt, A, B, C, D, E, F, G, H):
    sigma0 = right_rotate(A, 2) ^ right_rotate(A, 13) ^ right_rotate(A, 22)
    sigma1 = right_rotate(E, 6) ^ right_rotate(E, 11) ^ right_rotate(E, 25)
    ch = (E & F) ^ (~E & G)
    tmp1 = (H + sigma1 + ch + Kt + Wt) 
    maj = (A & B) ^ (A & C) ^ (B & C)
    tmp2 = (sigma0 + maj) 
    H = G
    G = F
    F = E
    E = (D + tmp1) % 2**32
    D = C
    C = B
    B = A
    A = (tmp1 + tmp2) % 2**32
    return A, B, C, D, E, F, G, H

In [192]:
for i in range(64):
    A, B, C, D, E, F, G, H = compression(W[i], constants[i], A, B, C, D, E, F, G, H)

In [193]:
compressed_chunk = [(x + y) % 2**32 for x, y in zip(digest, (A, B, C, D, E, F, G, H))]

In [194]:
test = [item.to_bytes(4, 'big') for item in compressed_chunk]
test2 = b"".join(test)
"".join(format(h, '02x') for h in test2)

'7a270a6f428c26310c647c4e46643c189cd254210cac94078c0bfed29f5f5e45'

In [113]:
"""This Python module is an implementation of the SHA-256 algorithm.
From https://github.com/keanemind/Python-SHA-256"""

K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]


def generate_hash(message: bytearray) -> bytearray:
    """Return a SHA-256 hash from the message passed.
    The argument should be a bytes, bytearray, or
    string object."""

    if isinstance(message, str):
        message = bytearray(message, 'ascii')
    elif isinstance(message, bytes):
        message = bytearray(message)
    elif not isinstance(message, bytearray):
        raise TypeError

    # Padding
    length = len(message) * 8  # len(message) is number of BYTES!!!
    message.append(0x80)
    while (len(message) * 8 + 64) % 512 != 0:
        message.append(0x00)

    message += length.to_bytes(8, 'big')  # pad to 8 bytes or 64 bits
    print(message)
    assert (len(message) * 8) % 512 == 0, "Padding did not complete properly!"

    # Parsing
    blocks = []  # contains 512-bit chunks of message
    for i in range(0, len(message), 64):  # 64 bytes is 512 bits
        blocks.append(message[i:i + 64])

    # Setting Initial Hash Value
    h0 = 0x6a09e667
    h1 = 0xbb67ae85
    h2 = 0x3c6ef372
    h3 = 0xa54ff53a
    h5 = 0x9b05688c
    h4 = 0x510e527f
    h6 = 0x1f83d9ab
    h7 = 0x5be0cd19

    # SHA-256 Hash Computation
    for message_block in blocks:
        # Prepare message schedule
        message_schedule = []
        for t in range(0, 64):
            if t <= 15:
                # adds the t'th 32 bit word of the block,
                # starting from leftmost word
                # 4 bytes at a time
                message_schedule.append(bytes(message_block[t * 4:(t * 4) + 4]))
            else:
                term1 = _sigma1(int.from_bytes(message_schedule[t - 2], 'big'))
                term2 = int.from_bytes(message_schedule[t - 7], 'big')
                term3 = _sigma0(int.from_bytes(message_schedule[t - 15], 'big'))
                term4 = int.from_bytes(message_schedule[t - 16], 'big')
                schedule = ((term1 + term2 + term3 + term4) % 2 ** 32)
                print('sigmal0', t, "{:032b}".format(term3))
                print('sigmal1', t, "{:032b}".format(term1))

                print('w', t, "{:032b}".format(schedule))
                # append a 4-byte byte object
                schedule = ((term1 + term2 + term3 + term4) % 2 ** 32).to_bytes(4, 'big')
                message_schedule.append(schedule)


        assert len(message_schedule) == 64

        # Initialize working variables
        a = h0
        b = h1
        c = h2
        d = h3
        e = h4
        f = h5
        g = h6
        h = h7

        # Iterate for t=0 to 63
        for t in range(64):
            t1 = ((h + _capsigma1(e) + _ch(e, f, g) + K[t] +
                   int.from_bytes(message_schedule[t], 'big')) % 2 ** 32)

            t2 = (_capsigma0(a) + _maj(a, b, c)) % 2 ** 32
            s0 = _capsigma0(a)
            s1 = _capsigma1(e)
            ch = _ch(e,f,g)
            maj = _maj(a,b,c)
            print('caosigma0', t, "{:032b}".format(s0))
            print('caosigma1', t, "{:032b}".format(s1))
            print('Ch', t, "{:032b}".format(ch))
            print('Maj', t, "{:032b}".format(maj))
            print('tmp1', t, "{:032b}".format(t1))
            print('tmp2', t, "{:032b}".format(t2))
            h = g
            g = f
            f = e
            e = (d + t1) % 2 ** 32
            d = c
            c = b
            b = a
            a = (t1 + t2) % 2 ** 32

        # Compute intermediate hash value
        h0 = (h0 + a) % 2 ** 32
        h1 = (h1 + b) % 2 ** 32
        h2 = (h2 + c) % 2 ** 32
        h3 = (h3 + d) % 2 ** 32
        h4 = (h4 + e) % 2 ** 32
        h5 = (h5 + f) % 2 ** 32
        h6 = (h6 + g) % 2 ** 32
        h7 = (h7 + h) % 2 ** 32

    return ((h0).to_bytes(4, 'big') + (h1).to_bytes(4, 'big') +
            (h2).to_bytes(4, 'big') + (h3).to_bytes(4, 'big') +
            (h4).to_bytes(4, 'big') + (h5).to_bytes(4, 'big') +
            (h6).to_bytes(4, 'big') + (h7).to_bytes(4, 'big'))


def _sigma0(num: int):
    """As defined in the specification."""
    num = (_rotate_right(num, 7) ^
           _rotate_right(num, 18) ^
           (num >> 3))
    return num


def _sigma1(num: int):
    """As defined in the specification."""
    num = (_rotate_right(num, 17) ^
           _rotate_right(num, 19) ^
           (num >> 10))
    return num


def _capsigma0(num: int):
    """As defined in the specification."""
    num = (_rotate_right(num, 2) ^
           _rotate_right(num, 13) ^
           _rotate_right(num, 22))
    return num


def _capsigma1(num: int):
    """As defined in the specification."""
    num = (_rotate_right(num, 6) ^
           _rotate_right(num, 11) ^
           _rotate_right(num, 25))
    return num


def _ch(x: int, y: int, z: int):
    """As defined in the specification."""
    return (x & y) ^ (~x & z)


def _maj(x: int, y: int, z: int):
    """As defined in the specification."""
    return (x & y) ^ (x & z) ^ (y & z)


def _rotate_right(num: int, shift: int, size: int = 32):
    """Rotate an integer right."""
    return (num >> shift) | (num << size - shift)


if __name__ == "__main__":
    print(generate_hash("Abc").hex())


bytearray(b'Abc\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18')
sigmal0 16 00000000000000000000000000000000
sigmal1 16 00000000000000000000000000000000
w 16 01000001011000100110001110000000
sigmal0 17 00000000000000000000000000000000
sigmal1 17 00000000000011110000000000000000
w 17 00000000000011110000000000000000
sigmal0 18 00000000000000000000000000000000
sigmal1 18 1010001001110101111101101000000111000000000101
w 18 01111101101000000111000000000101
sigmal0 19 00000000000000000000000000000000
sigmal1 19 11001100000000000000000001111000110
w 19 01100000000000000000001111000110
sigmal0 20 00000000000000000000000000000000
sigmal1 20 1100010110010000110110000111010111100101111000
w 20 00110110000111010111100101111000
sigmal0 21 00000000000000000000000000000000
sigmal1 21 1111000000000000000001